# **Рассчет ликвидной стоимости продажи автомобиля**

## Импорт библиотек, загрузка данных и общий просмотр

In [1]:
#%pip install -U scikit-learn

In [2]:
#%pip install seaborn

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
import numpy as np
import warnings
warnings.simplefilter("ignore")
import math
from statistics import mean
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [4]:
df = pd.DataFrame([{"km": 88000, "rubl": 1990000, "salesman": "дилер", "pts": 1, "equipment": "None", "color": "Белый", "location": "Калужская обл."},
                   {"km": 82000, "rubl": 1959000, "salesman": "дилер", "pts": 2, "equipment": "Trendline", "color": "Белый", "location": "Саратовская обл."},
                   {"km": 260280, "rubl": 1570000, "salesman": "дилер", "pts": 2, "equipment": "Winter Edition", "color": "Серый", "location": "Республика Карелия"},
                   {"km": 56500, "rubl": 1870000, "salesman": "дилер", "pts": 4, "equipment": "Trendline", "color": "Коричневый", "location": "Тюменская обл."},
                   {"km": 85800, "rubl": 2355000, "salesman": "дилер", "pts": 2, "equipment": "All Inclusive", "color": "Серый", "location": "Республика Крым"},
                   {"km": 93500, "rubl": 2000000, "salesman": "дилер", "pts": 2, "equipment": "City", "color": "Белый", "location": "Челябинская обл."},
                   {"km": 123000, "rubl": 2050000, "salesman": "хозяин", "pts": 3, "equipment": "Comfortline", "color": "Чёрный", "location": "Тюменская обл."},
                   {"km": 98500, "rubl": 2250000, "salesman": "хозяин", "pts": 2, "equipment": "Winter Edition", "color": "Белый", "location": "Челябинская обл."},
                   {"km": 117577, "rubl": 1984000, "salesman": "дилер", "pts": 2, "equipment": "Trendline", "color": "Белый", "location": "Калининградская обл."},
                   {"km": 114500, "rubl": 2340000, "salesman": "хозяин", "pts": 2, "equipment": "None", "color": "Чёрный", "location": "Республика Башкортостан"},
                   {"km": 178000, "rubl": 1669000, "salesman": "дилер", "pts": 3, "equipment": "Comfortline", "color": "Бежевый", "location": "Челябинская обл."},
                   {"km": 226056, "rubl": 1690000, "salesman": "хозяин", "pts": 3, "equipment": "Comfortline", "color": "Чёрный", "location": "Новгородская обл."},
                   {"km": 87000, "rubl": 2370000, "salesman": "хозяин", "pts": 4, "equipment": "Winter Edition", "color": "Чёрный", "location": "Псковская обл."},
                   {"km": 109500, "rubl": 2050000, "salesman": "хозяин", "pts": 1, "equipment": "Comfortline", "color": "Синий", "location": "Республика Коми"},
                   {"km": 31000, "rubl": 2699000, "salesman": "дилер", "pts": 3, "equipment": "None", "color": "Чёрный", "location": "Республика Хакасия"},
                   {"km": 103000, "rubl": 2100000, "salesman": "хозяин", "pts": 4, "equipment": "Trendline", "color": "Белый", "location": "Тюменская обл."},
                   {"km": 168000, "rubl": None, "salesman": "хозяин", "pts": 2, "equipment": "Trendline", "color": "Чёрный", "location": "Воронежская обл."}])
df

,km,rubl,salesman,pts,equipment,color,location
0,88000,1990000.0,дилер,1,None,Белый,Калужская обл.
1,82000,1959000.0,дилер,2,Trendline,Белый,Саратовская обл.
2,260280,1570000.0,дилер,2,Winter Edition,Серый,Республика Карелия
3,56500,1870000.0,дилер,4,Trendline,Коричневый,Тюменская обл.
4,85800,2355000.0,дилер,2,All Inclusive,Серый,Республика Крым
5,93500,2000000.0,дилер,2,City,Белый,Челябинская обл.
6,123000,2050000.0,хозяин,3,Comfortline,Чёрный,Тюменская обл.
7,98500,2250000.0,хозяин,2,Winter Edition,Белый,Челябинская обл.
8,117577,1984000.0,дилер,2,Trendline,Белый,Калининградская обл.
9,114500,2340000.0,хозяин,2,None,Чёрный,Республика Башкортостан


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   km         17 non-null     int64  
 1   rubl       16 non-null     float64
 2   salesman   17 non-null     object 
 3   pts        17 non-null     int64  
 4   equipment  17 non-null     object 
 5   color      17 non-null     object 
 6   location   17 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 1.1+ KB


In [6]:
my_km = df.iloc[16]
my_km

km                     168000
rubl                      NaN
salesman               хозяин
pts                         2
equipment           Trendline
color                  Чёрный
location     Воронежская обл.
Name: 16, dtype: object

In [7]:
#df = df.drop(16)

## Функция для вычисления sMAPE

In [8]:
def sMAPE(y_true,y_pred):
    error = np.mean(abs(y_true - y_pred)/((abs(y_true)+abs(y_pred))/2))*100
    return error

In [9]:
bst_model = None

## Разделение выборки на тестовую, тренировочную и валидационную, а также масштабирование и кодирование

In [10]:
features_dumm = df.drop(columns=['rubl'])
features_Ord = df.drop(columns=['rubl'])

In [11]:
target_dumm = df['rubl']
target_Ord = df['rubl']

In [12]:
features_dumm

,km,salesman,pts,equipment,color,location
0,88000,дилер,1,None,Белый,Калужская обл.
1,82000,дилер,2,Trendline,Белый,Саратовская обл.
2,260280,дилер,2,Winter Edition,Серый,Республика Карелия
3,56500,дилер,4,Trendline,Коричневый,Тюменская обл.
4,85800,дилер,2,All Inclusive,Серый,Республика Крым
5,93500,дилер,2,City,Белый,Челябинская обл.
6,123000,хозяин,3,Comfortline,Чёрный,Тюменская обл.
7,98500,хозяин,2,Winter Edition,Белый,Челябинская обл.
8,117577,дилер,2,Trendline,Белый,Калининградская обл.
9,114500,хозяин,2,None,Чёрный,Республика Башкортостан


In [13]:
features_dumm.shape

(17, 6)

In [14]:
target_dumm.shape

(17,)

In [15]:
# прямое кодирование
features_dumm = pd.get_dummies(features_dumm, columns=['salesman', 'equipment', 'color', 'location'], drop_first=True)

In [16]:
features_dumm

,km,pts,salesman_хозяин,equipment_City,equipment_Comfortline,equipment_None,equipment_Trendline,equipment_Winter Edition,color_Белый,color_Коричневый,...,location_Новгородская обл.,location_Псковская обл.,location_Республика Башкортостан,location_Республика Карелия,location_Республика Коми,location_Республика Крым,location_Республика Хакасия,location_Саратовская обл.,location_Тюменская обл.,location_Челябинская обл.
0,88000,1,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,82000,2,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,True,False,False
2,260280,2,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
3,56500,4,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False
4,85800,2,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
5,93500,2,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
6,123000,3,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
7,98500,2,True,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,True
8,117577,2,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
9,114500,2,True,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [17]:
# порядковое кодирование
ord_enc = OrdinalEncoder()

In [18]:
features_Ord[['salesman', 'equipment', 'color', 'location']] = ord_enc.fit_transform(features_Ord[['salesman', 'equipment', 'color', 'location']])

In [19]:
# масштабирование признаков features_dumm
numeric = ['km', 'pts']

scaler = StandardScaler()
scaler.fit(features_dumm[numeric])
features_dumm[numeric] = scaler.transform(features_dumm[numeric])

In [20]:
features_dumm

,km,pts,salesman_хозяин,equipment_City,equipment_Comfortline,equipment_None,equipment_Trendline,equipment_Winter Edition,color_Белый,color_Коричневый,...,location_Новгородская обл.,location_Псковская обл.,location_Республика Башкортостан,location_Республика Карелия,location_Республика Коми,location_Республика Крым,location_Республика Хакасия,location_Саратовская обл.,location_Тюменская обл.,location_Челябинская обл.
0,-0.544868,-1.607061,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,-0.650484,-0.514259,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,True,False,False
2,2.487721,-0.514259,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
3,-1.099352,1.671343,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False
4,-0.583594,-0.514259,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
5,-0.448053,-0.514259,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
6,0.071226,0.578542,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
7,-0.360040,-0.514259,True,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,True
8,-0.024234,-0.514259,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
9,-0.078397,-0.514259,True,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [21]:
target_dumm

0     1990000.0
1     1959000.0
2     1570000.0
3     1870000.0
4     2355000.0
5     2000000.0
6     2050000.0
7     2250000.0
8     1984000.0
9     2340000.0
10    1669000.0
11    1690000.0
12    2370000.0
13    2050000.0
14    2699000.0
15    2100000.0
16          NaN
Name: rubl, dtype: float64

In [22]:
my_features_dumm = features_dumm.iloc[16]
my_features_dumm

km                                  0.863346
pts                                -0.514259
salesman_хозяин                         True
equipment_City                         False
equipment_Comfortline                  False
equipment_None                         False
equipment_Trendline                     True
equipment_Winter Edition               False
color_Белый                            False
color_Коричневый                       False
color_Серый                            False
color_Синий                            False
color_Чёрный                            True
location_Калининградская обл.          False
location_Калужская обл.                False
location_Новгородская обл.             False
location_Псковская обл.                False
location_Республика Башкортостан       False
location_Республика Карелия            False
location_Республика Коми               False
location_Республика Крым               False
location_Республика Хакасия            False
location_С

In [23]:
features_dumm = features_dumm.drop(16)
target_dumm = target_dumm.drop(16)

In [24]:
# масштабирование признаков features_Ord
numeric = ['km', 'pts']

scaler = StandardScaler()
scaler.fit(features_Ord[numeric])
features_Ord[numeric] = scaler.transform(features_Ord[numeric])

In [25]:
features_Ord

,km,salesman,pts,equipment,color,location
0,-0.544868,0.0,-1.607061,3.0,1.0,2.0
1,-0.650484,0.0,-0.514259,4.0,1.0,10.0
2,2.487721,0.0,-0.514259,5.0,3.0,6.0
3,-1.099352,0.0,1.671343,4.0,2.0,11.0
4,-0.583594,0.0,-0.514259,0.0,3.0,8.0
5,-0.448053,0.0,-0.514259,1.0,1.0,12.0
6,0.071226,1.0,0.578542,2.0,5.0,11.0
7,-0.360040,1.0,-0.514259,5.0,1.0,12.0
8,-0.024234,0.0,-0.514259,4.0,1.0,1.0
9,-0.078397,1.0,-0.514259,3.0,5.0,5.0


In [26]:
target_Ord

0     1990000.0
1     1959000.0
2     1570000.0
3     1870000.0
4     2355000.0
5     2000000.0
6     2050000.0
7     2250000.0
8     1984000.0
9     2340000.0
10    1669000.0
11    1690000.0
12    2370000.0
13    2050000.0
14    2699000.0
15    2100000.0
16          NaN
Name: rubl, dtype: float64

In [27]:
my_features_Ord = features_Ord.iloc[16]
my_features_Ord

km           0.863346
salesman     1.000000
pts         -0.514259
equipment    4.000000
color        5.000000
location     0.000000
Name: 16, dtype: float64

In [28]:
features_Ord = features_Ord.drop(16)
target_Ord = target_Ord.drop(16)

In [29]:
# деление на обучающую и валидационно-тестовую выборку (60:40)
features_train_dumm, features_valid_test_dumm, target_train_dumm, target_valid_test_dumm = train_test_split(
    features_dumm, target_dumm, test_size=0.4, random_state=12345)

In [30]:
# деление валидационно-тестовой выборки (50:50)
features_valid_dumm, features_test_dumm, target_valid_dumm, target_test_dumm = train_test_split(
    features_valid_test_dumm, target_valid_test_dumm, test_size=0.5, random_state=12345)

In [31]:
# деление на обучающую и валидационно-тестовую выборку (60:40)
features_train_Ord, features_valid_test_Ord, target_train_Ord, target_valid_test_Ord = train_test_split(
    features_Ord, target_Ord, test_size=0.4, random_state=12345)

In [32]:
# деление валидационно-тестовой выборки (50:50)
features_valid_Ord, features_test_Ord, target_valid_Ord, target_test_Ord = train_test_split(
    features_valid_test_Ord, target_valid_test_Ord, test_size=0.5, random_state=12345)

In [33]:
features_dumm.head(3)

,km,pts,salesman_хозяин,equipment_City,equipment_Comfortline,equipment_None,equipment_Trendline,equipment_Winter Edition,color_Белый,color_Коричневый,...,location_Новгородская обл.,location_Псковская обл.,location_Республика Башкортостан,location_Республика Карелия,location_Республика Коми,location_Республика Крым,location_Республика Хакасия,location_Саратовская обл.,location_Тюменская обл.,location_Челябинская обл.
0,-0.544868,-1.607061,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,-0.650484,-0.514259,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,True,False,False
2,2.487721,-0.514259,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False


In [34]:
features_Ord.head(3)

,km,salesman,pts,equipment,color,location
0,-0.544868,0.0,-1.607061,3.0,1.0,2.0
1,-0.650484,0.0,-0.514259,4.0,1.0,10.0
2,2.487721,0.0,-0.514259,5.0,3.0,6.0


In [35]:
features_Ord.shape

(16, 6)

In [36]:
features_dumm.shape

(16, 25)

# Вариант решения №1

## Обучение моделей

### LinearRegression

In [37]:
# на данных с прямым кодированием
%%time
# расчет для LinearRegression
dumm_model_LinearRegression = LinearRegression()
dumm_model_LinearRegression.fit(features_train_dumm, target_train_dumm)
y_true = target_valid_dumm
y_pred = pd.DataFrame(dumm_model_LinearRegression.predict(features_valid_dumm))

result = mean_squared_error(y_true, y_pred)**0.5
result_1 = r2_score(y_true, y_pred)

print("Модель:", dumm_model_LinearRegression)
print("RMSE:", round(result, 1))
print("R2:", round(result_1, 3))

Модель: LinearRegression()
RMSE: 145291.1
R2: 0.607
CPU times: user 20.6 ms, sys: 2.21 ms, total: 22.8 ms
Wall time: 43.8 ms


In [38]:
# на данных с порядковым кодированием
%%time
# расчет для LinearRegression
model_LinearRegression = LinearRegression()
model_LinearRegression.fit(features_train_Ord, target_train_Ord)
y_true = target_valid_Ord
y_pred = pd.DataFrame(model_LinearRegression.predict(features_valid_Ord))

result = mean_squared_error(y_true, y_pred)**0.5
result_1 = r2_score(y_true, y_pred)

print("Модель:", model_LinearRegression)
print("RMSE:", round(result, 1))
print("R2:", round(result_1, 3))

Модель: LinearRegression()
RMSE: 441702.2
R2: -2.634
CPU times: user 18 ms, sys: 928 µs, total: 18.9 ms
Wall time: 29.7 ms


### RandomForestRegressor

In [39]:
# на данных с прямым кодированием
%%time
# подбор гиперпараметров: глубины дерева, числа деревьев на основе RMSE
dumm_best_model_RandomForestRegressor = None
best_result = 100000000
best_result_1 = 0
for depth in range(1, 6, 1):
  for est in range(1, 6, 1):
    model = RandomForestRegressor(n_estimators=est, max_depth=depth, random_state=12345)
    model.fit(features_train_dumm, target_train_dumm)

    y_true = target_valid_dumm
    y_pred = pd.DataFrame(model.predict(features_valid_dumm))

    result = mean_squared_error(y_true, y_pred)**0.5
    result_1 = r2_score(y_true, y_pred)
    if result < best_result:
      dumm_best_model_RandomForestRegressor = model
      best_result = result
      best_result_1 = result_1

print("Лучшие параметры модели:", dumm_best_model_RandomForestRegressor)
print("RMSE:", round(best_result, 1))
print("R2:", round(result_1, 3))

Лучшие параметры модели: RandomForestRegressor(max_depth=2, n_estimators=2, random_state=12345)
RMSE: 150166.7
R2: 0.309
CPU times: user 515 ms, sys: 9.94 ms, total: 525 ms
Wall time: 845 ms


In [40]:
# на данных с порядковым кодированием
%%time
# подбор гиперпараметров: глубины дерева, числа деревьев на основе RMSE
best_model_RandomForestRegressor = None
best_result = 100000000
best_result_1 = 0
for depth in range(1, 6, 1):
  for est in range(1, 6, 1):
    model = RandomForestRegressor(n_estimators=est, max_depth=depth, random_state=12345)
    model.fit(features_train_Ord, target_train_Ord)

    y_true = target_valid_Ord
    y_pred = pd.DataFrame(model.predict(features_valid_Ord))

    result = mean_squared_error(y_true, y_pred)**0.5
    result_1 = r2_score(y_true, y_pred)
    if result < best_result:
      best_model_RandomForestRegressor = model
      best_result = result
      best_result_1 = result_1

print("Лучшие параметры модели:", best_model_RandomForestRegressor)
print("RMSE:", round(best_result, 1))
print("R2:", round(result_1, 3))

Лучшие параметры модели: RandomForestRegressor(max_depth=2, n_estimators=2, random_state=12345)
RMSE: 352019.5
R2: -1.963
CPU times: user 467 ms, sys: 3.22 ms, total: 470 ms
Wall time: 630 ms


In [41]:
# на данных с прямым кодированием
%%time
# подбор гиперпараметров: глубины дерева, минимального числа samples_split на основе RMSE
dumm_best_model_DecisionTreeRegressor = None
best_result = 100000000
best_result_1 = 0
for depth in range(1, 6, 1):
  for spl in range(2, 7, 1):
    model = DecisionTreeRegressor(max_depth=depth, min_samples_split=spl, random_state=12345)
    model.fit(features_train_dumm, target_train_dumm)

    y_true = target_valid_dumm
    y_pred = pd.DataFrame(model.predict(features_valid_dumm))

    result = mean_squared_error(y_true, y_pred)**0.5
    result_1 = r2_score(y_true, y_pred)
    if result < best_result:
      dumm_best_model_DecisionTreeRegressor = model
      best_result = result
      best_result_1 = result_1

print("Лучшие параметры модели:", dumm_best_model_DecisionTreeRegressor)
print("RMSE:", round(best_result, 1))
print("R2:", round(result_1, 3))

Лучшие параметры модели: DecisionTreeRegressor(max_depth=2, random_state=12345)
RMSE: 83572.3
R2: 0.396
CPU times: user 292 ms, sys: 5.36 ms, total: 298 ms
Wall time: 501 ms


In [42]:
# на данных с порядковым кодированием
%%time
# подбор гиперпараметров: глубины дерева, минимального числа samples_split на основе RMSE
best_model_DecisionTreeRegressor = None
best_result = 100000000
best_result_1 = 0
for depth in range(1, 6, 1):
  for spl in range(2, 7, 1):
    model = DecisionTreeRegressor(max_depth=depth, min_samples_split=spl, random_state=12345)
    model.fit(features_train_Ord, target_train_Ord)

    y_true = target_valid_Ord
    y_pred = pd.DataFrame(model.predict(features_valid_Ord))

    result = mean_squared_error(y_true, y_pred)**0.5
    result_1 = r2_score(y_true, y_pred)
    if result < best_result:
      best_model_DecisionTreeRegressor = model
      best_result = result
      best_result_1 = result_1

print("Лучшие параметры модели:", best_model_DecisionTreeRegressor)
print("RMSE:", round(best_result, 1))
print("R2:", round(result_1, 3))

Лучшие параметры модели: DecisionTreeRegressor(max_depth=5, random_state=12345)
RMSE: 92088.7
R2: 0.634
CPU times: user 250 ms, sys: 5.12 ms, total: 255 ms
Wall time: 334 ms


### DummyRegressor

In [43]:
# на данных с прямым кодированием
%%time
# подбор гиперпараметров: ['mean', 'median'] на основе RMSE
dumm_best_model_DummyRegressor = None
best_result = 100000000
best_result_1 = 0
list_strategy = ['mean', 'median']
for strat in list_strategy:
  model = DummyRegressor(strategy=strat)
  model.fit(features_train_dumm, target_train_dumm)

  y_true = target_valid_dumm
  y_pred = pd.DataFrame(model.predict(features_valid_dumm))

  result = mean_squared_error(y_true, y_pred)**0.5
  result_1 = r2_score(y_true, y_pred)
  if result < best_result:
    dumm_best_model_DummyRegressor = model
    best_result = result
    best_result_1 = result_1

print("Лучшие параметры модели:", dumm_best_model_DummyRegressor)
print("RMSE:", round(best_result, 1))
print("R2:", round(result_1, 3))

Лучшие параметры модели: DummyRegressor(strategy='median')
RMSE: 237767.4
R2: -0.053
CPU times: user 16.1 ms, sys: 41 µs, total: 16.2 ms
Wall time: 42.6 ms


In [44]:
# на данных с порядковым кодированием
%%time
# подбор гиперпараметров: ['mean', 'median'] на основе RMSE
best_model_DummyRegressor = None
best_result = 100000000
best_result_1 = 0
list_strategy = ['mean', 'median']
for strat in list_strategy:
  model = DummyRegressor(strategy=strat)
  model.fit(features_train_Ord, target_train_Ord)

  y_true = target_valid_Ord
  y_pred = pd.DataFrame(model.predict(features_valid_Ord))

  result = mean_squared_error(y_true, y_pred)**0.5
  result_1 = r2_score(y_true, y_pred)
  if result < best_result:
    best_model_DummyRegressor = model
    best_result = result
    best_result_1 = result_1

print("Лучшие параметры модели:", best_model_DummyRegressor)
print("RMSE:", round(best_result, 1))
print("R2:", round(result_1, 3))

Лучшие параметры модели: DummyRegressor(strategy='median')
RMSE: 237767.4
R2: -0.053
CPU times: user 15 ms, sys: 986 µs, total: 16 ms
Wall time: 52.2 ms


### LGBMRegressor

In [45]:
# на данных с прямым кодированием
%%time
# подбор гиперпараметров: глубины дерева, числа деревьев на основе RMSE
dumm_best_model_LGBMRegressor = None
best_result = 100000000
best_result_1 = 0
for depth in range(1, 6, 1):
  for est in range(1, 6, 1):
    model = LGBMRegressor(n_estimators=est, max_depth=depth, random_state=12345)
    model.fit(features_train_dumm, target_train_dumm)

    y_true = target_valid_dumm
    y_pred = pd.DataFrame(model.predict(features_valid_dumm))

    result = mean_squared_error(y_true, y_pred)**0.5
    result_1 = r2_score(y_true, y_pred)
    if result < best_result:
      dumm_best_model_LGBMRegressor = model
      best_result = result
      best_result_1 = result_1

print("Лучшие параметры модели:", dumm_best_model_LGBMRegressor)
print("RMSE:", round(best_result, 1))
print("R2:", round(result_1, 3))

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 9, number of used features: 0
[LightGBM] [Info] Start training from score 2082444.444444
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 9, number of used features: 0
[Li

In [46]:
# на данных с порядковым кодированием
%%time
# подбор гиперпараметров: глубины дерева, числа деревьев на основе RMSE
best_model_LGBMRegressor = None
best_result = 100000000
best_result_1 = 0
for depth in range(1, 6, 1):
  for est in range(1, 6, 1):
    model = LGBMRegressor(n_estimators=est, max_depth=depth, random_state=12345)
    model.fit(features_train_Ord, target_train_Ord)

    y_true = target_valid_Ord
    y_pred = pd.DataFrame(model.predict(features_valid_Ord))

    result = mean_squared_error(y_true, y_pred)**0.5
    result_1 = r2_score(y_true, y_pred)
    if result < best_result:
      best_model_LGBMRegressor = model
      best_result = result
      best_result_1 = result_1

print("Лучшие параметры модели:", best_model_LGBMRegressor)
print("RMSE:", round(best_result, 1))
print("R2:", round(result_1, 3))

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 9, number of used features: 0
[LightGBM] [Info] Start training from score 2082444.444444
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 9, number of used features: 0
[LightGBM] [Info] Start training from score 2082444.444444
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements


## Анализ моделей

In [47]:
dumm_list_model = [dumm_model_LinearRegression, dumm_best_model_RandomForestRegressor, dumm_best_model_DecisionTreeRegressor, dumm_best_model_DummyRegressor, dumm_best_model_LGBMRegressor]

In [48]:
# поиск наилучшей модели на ТЕСТОВОЙ выборке
best_model = None
best_result = 100000000
for mod in dumm_list_model:
  test_predictions = mod.predict(features_test_dumm)
  y_true = target_test_dumm
  y_pred = pd.DataFrame(mod.predict(features_test_dumm))
  result = mean_squared_error(y_true, y_pred)**0.5
  if result < best_result:
    best_result = result
    best_model = mod

print("Наилучшая модель на ТЕСТОВОЙ выборке:", best_model)
print("RMSE наилучшей модели =", round(best_result, 1))

Наилучшая модель на ТЕСТОВОЙ выборке: DummyRegressor(strategy='median')
RMSE наилучшей модели = 188915.3


In [49]:
list_model = [model_LinearRegression, best_model_RandomForestRegressor, best_model_DecisionTreeRegressor, best_model_DummyRegressor, best_model_LGBMRegressor]

In [50]:
# поиск наилучшей модели на ТЕСТОВОЙ выборке
best_model = None
best_result = 100000000
for mod in list_model:
  test_predictions = mod.predict(features_test_Ord)
  y_true = target_test_Ord
  y_pred = pd.DataFrame(mod.predict(features_test_Ord))
  result = mean_squared_error(y_true, y_pred)**0.5
  if result < best_result:
    best_result = result
    best_model = mod

print("Наилучшая модель на ТЕСТОВОЙ выборке:", best_model)
print("RMSE наилучшей модели =", round(best_result, 1))

Наилучшая модель на ТЕСТОВОЙ выборке: DummyRegressor(strategy='median')
RMSE наилучшей модели = 188915.3


## Вывод цены и точности предсказания

In [51]:
bst_model = DummyRegressor(strategy='median').fit(features_dumm, target_dumm)
predicted_prices = bst_model.predict(my_features_dumm)
print("Предсказанная цена автомобиля =", np.round(predicted_prices[0], 0))

result = sMAPE(target_dumm, pd.DataFrame(bst_model.predict(features_dumm)))
print("Точность предсказания модели:", 100 - round(result, 2), "%")

Предсказанная цена автомобиля = 2025000.0
Точность предсказания модели: 98.26 %


In [52]:
bst_model = DummyRegressor(strategy='median').fit(features_Ord, target_Ord)
predicted_prices = bst_model.predict(my_features_Ord)
print("Предсказанная цена автомобиля =", np.round(predicted_prices[0], 0))

result = sMAPE(target_Ord, pd.DataFrame(bst_model.predict(features_Ord)))
print("Точность предсказания модели:", 100 - round(result, 2), "%")

Предсказанная цена автомобиля = 2025000.0
Точность предсказания модели: 98.26 %


# Вариант решения №2

## Обучение и проверка модели

In [53]:
# функция вывода на печать предсказанного и истинного значения цены автомобиля, а также параметров, оценивающих качество модели
def param_model(features, target):
  # деление данных на обучающую и валидационную выборки в соотношении 75:25
  features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
  # обучение моли линейной регрессии на данных региона
  # вывод предсказанного средней цены автомобиля
  # вывод метрик качества модели линейной регрессии
  model = LinearRegression().fit(features_train, target_train)
  predicted_valid = model.predict(features_valid)
  mse = mean_squared_error(target_valid, predicted_valid)
  result = sMAPE(target_valid, pd.DataFrame(model.predict(features_valid)))


  print("Предсказанная средняя цена автомобиля =", round(predicted_valid.mean(), 1))
  print()
  print("RMSE =", round(mse ** 0.5, 1))
  print("MSE =", round(mse, 1))
  print("R2 =", round(r2_score(target_valid, predicted_valid), 1))
  print("MAE =", round(mean_absolute_error(target_valid, predicted_valid), 1))
  print("Точность предсказания модели по sMAPE:", 100 - round(result, 2), "%")
  print()
  print("Истинное среднее значение цены автомобиля =", round(target.mean(), 1))

In [54]:
param_model(features_dumm, target_dumm)

Предсказанная средняя цена автомобиля = 2028698.9

RMSE = 152545.5
MSE = 23270138985.5
R2 = 0.4
MAE = 131048.0
Точность предсказания модели по sMAPE: 91.9 %

Истинное среднее значение цены автомобиля = 2059125.0


In [55]:
param_model(features_Ord, target_Ord)

Предсказанная средняя цена автомобиля = 2076051.7

RMSE = 383005.0
MSE = 146692806494.0
R2 = -2.7
MAE = 363539.8
Точность предсказания модели по sMAPE: 89.01 %

Истинное среднее значение цены автомобиля = 2059125.0


In [56]:
my_features_dumm

km                                  0.863346
pts                                -0.514259
salesman_хозяин                         True
equipment_City                         False
equipment_Comfortline                  False
equipment_None                         False
equipment_Trendline                     True
equipment_Winter Edition               False
color_Белый                            False
color_Коричневый                       False
color_Серый                            False
color_Синий                            False
color_Чёрный                            True
location_Калининградская обл.          False
location_Калужская обл.                False
location_Новгородская обл.             False
location_Псковская обл.                False
location_Республика Башкортостан       False
location_Республика Карелия            False
location_Республика Коми               False
location_Республика Крым               False
location_Республика Хакасия            False
location_С

In [57]:
features_dumm

,km,pts,salesman_хозяин,equipment_City,equipment_Comfortline,equipment_None,equipment_Trendline,equipment_Winter Edition,color_Белый,color_Коричневый,...,location_Новгородская обл.,location_Псковская обл.,location_Республика Башкортостан,location_Республика Карелия,location_Республика Коми,location_Республика Крым,location_Республика Хакасия,location_Саратовская обл.,location_Тюменская обл.,location_Челябинская обл.
0,-0.544868,-1.607061,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,-0.650484,-0.514259,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,True,False,False
2,2.487721,-0.514259,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
3,-1.099352,1.671343,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False
4,-0.583594,-0.514259,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
5,-0.448053,-0.514259,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
6,0.071226,0.578542,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
7,-0.360040,-0.514259,True,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,True
8,-0.024234,-0.514259,False,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False
9,-0.078397,-0.514259,True,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [58]:
np.array([my_features_dumm])

array([[0.8633460971129019, -0.51425947722658, True, False, False, False,
        True, False, False, False, False, False, True, False, False,
        False, False, False, False, False, False, False, False, False,
        False]], dtype=object)

In [59]:
bst_model = LinearRegression().fit(features_dumm, target_dumm)
predicted_prices = bst_model.predict(np.array([my_features_dumm]))
print("Предсказанная цена автомобиля =", np.round(predicted_prices[0], 0))

result = sMAPE(target_dumm, pd.DataFrame(bst_model.predict(features_dumm)))
print("Точность предсказания модели:", 100 - round(result, 2), "%")

Предсказанная цена автомобиля = 1962941.0
Точность предсказания модели: 89.36 %


In [60]:
bst_model = LinearRegression().fit(features_Ord, target_Ord)
predicted_prices = bst_model.predict(np.array([my_features_Ord]))
print("Предсказанная цена автомобиля =", np.round(predicted_prices[0], 0))

result = sMAPE(target_Ord, pd.DataFrame(bst_model.predict(features_Ord)))
print("Точность предсказания модели:", 100 - round(result, 2), "%")

Предсказанная цена автомобиля = 2049448.0
Точность предсказания модели: 89.53999999999999 %


# Вариант решения №3

### Обучение моделей и оценка их качества кросс-валидацией

In [61]:
# деление данных на обучающую и валидационную выборки в соотношении 73:27
dumm_features_train, dumm_features_valid, dumm_target_train, dumm_target_valid = train_test_split(
    features_dumm, target_dumm, test_size=0.27, random_state=12345)

In [62]:
# деление данных на обучающую и валидационную выборки в соотношении 73:27
Ord_features_train, Ord_features_valid, Ord_target_train, Ord_target_valid = train_test_split(
    features_Ord, target_Ord, test_size=0.27, random_state=12345)

In [63]:
# функция поиска лучшей модели на кросс-валидации в Python (cv=cv_num)
def search_best_model(list_model, features, target, cv_num):
  best_model = None
  score = 100
  for model in list_model:
    scores = []
    sample_size = int(len(features)/cv_num)
    for i in range(0, len(features), sample_size):
      valid_indexes = features.iloc[i: i + sample_size].index
      train_indexes = (features.iloc[:i] + features.iloc[i + sample_size : len(features)]).index

      features_train = features.iloc[train_indexes]
      features_valid = features.iloc[valid_indexes]
      target_train = target.iloc[train_indexes]
      target_valid = target.iloc[valid_indexes]

      model = model.fit(features_train, target_train)

      y_true = target_valid
      y_pred = pd.DataFrame(model.predict(features_valid))

      fin_sMAPE = sMAPE(y_true, y_pred)
      scores.append(fin_sMAPE)
    scores = pd.Series(scores)
    result = scores.mean()
    if result < score:
      score = result
      best_model = model
  print("Среднее итоговое sMAPE лучшей модели на тестовом датафрейме:", round(score, 2), "%)")
  print("Лучшая модель:", best_model)
  print("Точность предсказания лучшей модели:", 100 - round(score, 2), "%")

### LinearRegression

In [64]:
# расчет sMAPE для LinearRegression
model = LinearRegression()
model.fit(dumm_features_train, dumm_target_train)
y_true = dumm_target_valid
y_pred = pd.DataFrame(model.predict(dumm_features_valid))

result = sMAPE(y_true, y_pred)

print("Модель:", model)
print("Итоговое sMAPE:", 100 - round(result, 1), "%")

Модель: LinearRegression()
Итоговое sMAPE: 90.5 %


In [65]:
dumm_model_LinearRegression = model

In [66]:
# расчет sMAPE для LinearRegression
model = LinearRegression()
model.fit(Ord_features_train, Ord_target_train)
y_true = Ord_target_valid
y_pred = pd.DataFrame(model.predict(Ord_features_valid))

result = sMAPE(y_true, y_pred)

print("Модель:", model)
print("Итоговое sMAPE:", 100 - round(result, 1), "%")

Модель: LinearRegression()
Итоговое sMAPE: 89.9 %


In [67]:
Ord_model_LinearRegression = model

### RandomForestRegressor

In [68]:
# подбор гиперпараметров: глубины дерева, числа деревьев на основе sMAPE
best_model_RandomForestRegressor = None
best_result = 100
for depth in range(1, 5, 1):
  for est in range(1, 5, 1):
    model = RandomForestRegressor(n_estimators=est, max_depth=depth, random_state=12345)
    model.fit(dumm_features_train, dumm_target_train)

    y_true = dumm_target_valid
    y_pred = pd.DataFrame(model.predict(dumm_features_valid))

    result = sMAPE(y_true, y_pred)
    if result < best_result:
      dumm_best_model_RandomForestRegressor = model
      best_result = result

print("Лучшие параметры модели:", dumm_best_model_RandomForestRegressor)
print("Итоговое sMAPE:", 100 - round(best_result, 1), "%")

Лучшие параметры модели: RandomForestRegressor(max_depth=1, n_estimators=4, random_state=12345)
Итоговое sMAPE: 93.5 %


In [69]:
# подбор гиперпараметров: глубины дерева, числа деревьев на основе sMAPE
best_model_RandomForestRegressor = None
best_result = 100
for depth in range(1, 5, 1):
  for est in range(1, 5, 1):
    model = RandomForestRegressor(n_estimators=est, max_depth=depth, random_state=12345)
    model.fit(Ord_features_train, Ord_target_train)

    y_true = Ord_target_valid
    y_pred = pd.DataFrame(model.predict(Ord_features_valid))

    result = sMAPE(y_true, y_pred)
    if result < best_result:
      Ord_best_model_RandomForestRegressor = model
      best_result = result

print("Лучшие параметры модели:", Ord_best_model_RandomForestRegressor)
print("Итоговое sMAPE:", 100 - round(best_result, 1), "%")

Лучшие параметры модели: RandomForestRegressor(max_depth=1, n_estimators=4, random_state=12345)
Итоговое sMAPE: 93.5 %


### DecisionTreeRegressor

In [70]:
# подбор гиперпараметров: глубины дерева, минимального числа samples_split на основе sMAPE
best_model_DecisionTreeRegressor = None
best_result = 100
for depth in range(1, 5, 1):
  for spl in range(2, 6, 1):
    model = DecisionTreeRegressor(max_depth=depth, min_samples_split=spl, random_state=12345)
    model.fit(dumm_features_train, dumm_target_train)

    y_true = dumm_target_valid
    y_pred = pd.DataFrame(model.predict(dumm_features_valid))

    result = sMAPE(y_true, y_pred)
    if result < best_result:
      dumm_best_model_DecisionTreeRegressor = model
      best_result = result

print("Лучшие параметры модели:", dumm_best_model_DecisionTreeRegressor)
print("Итоговое sMAPE:", 100 - round(best_result, 1), "%")

Лучшие параметры модели: DecisionTreeRegressor(max_depth=3, random_state=12345)
Итоговое sMAPE: 93.3 %


In [71]:
# подбор гиперпараметров: глубины дерева, минимального числа samples_split на основе sMAPE
best_model_DecisionTreeRegressor = None
best_result = 100
for depth in range(1, 5, 1):
  for spl in range(2, 6, 1):
    model = DecisionTreeRegressor(max_depth=depth, min_samples_split=spl, random_state=12345)
    model.fit(Ord_features_train, Ord_target_train)

    y_true = Ord_target_valid
    y_pred = pd.DataFrame(model.predict(Ord_features_valid))

    result = sMAPE(y_true, y_pred)
    if result < best_result:
      Ord_best_model_DecisionTreeRegressor = model
      best_result = result

print("Лучшие параметры модели:", Ord_best_model_DecisionTreeRegressor)
print("Итоговое sMAPE:", 100 - round(best_result, 1), "%")

Лучшие параметры модели: DecisionTreeRegressor(max_depth=1, random_state=12345)
Итоговое sMAPE: 87.4 %


### Выбор лучшей модели среди лучших

In [72]:
# лист из претендентов на лучшую модель Ord
list_Ord_model = [Ord_model_LinearRegression, Ord_best_model_RandomForestRegressor, Ord_best_model_DecisionTreeRegressor]

In [73]:
search_best_model(list_Ord_model, features_Ord, target_Ord, 3)

Среднее итоговое sMAPE лучшей модели на тестовом датафрейме: 9.13 %)
Лучшая модель: LinearRegression()
Точность предсказания лучшей модели: 90.87 %


In [74]:
bst_model = LinearRegression().fit(features_Ord, target_Ord)
predicted_prices = bst_model.predict(np.array([my_features_Ord]))

print("Предсказанная цена автомобиля =", np.round(predicted_prices[0], 0))

result = sMAPE(target_Ord, pd.DataFrame(bst_model.predict(features_Ord)))
print("Точность предсказания модели:", 100 - round(result, 2), "%")

Предсказанная цена автомобиля = 2049448.0
Точность предсказания модели: 89.53999999999999 %


In [75]:
# лист из претендентов на лучшую модель dumm
list_dumm_model = [dumm_model_LinearRegression, dumm_best_model_RandomForestRegressor, dumm_best_model_DecisionTreeRegressor]

In [76]:
search_best_model(list_dumm_model, features_dumm, target_dumm, 3)

Среднее итоговое sMAPE лучшей модели на тестовом датафрейме: 12.69 %)
Лучшая модель: LinearRegression()
Точность предсказания лучшей модели: 87.31 %


In [77]:
bst_model = LinearRegression().fit(features_dumm, target_dumm)
predicted_prices = bst_model.predict(np.array([my_features_dumm]))

print("Предсказанная цена автомобиля =", np.round(predicted_prices[0], 0))

result = sMAPE(target_dumm, pd.DataFrame(bst_model.predict(features_dumm)))
print("Точность предсказания модели:", 100 - round(result, 2), "%")

Предсказанная цена автомобиля = 1962941.0
Точность предсказания модели: 89.36 %


# Вариант решения №4

In [78]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [87]:
X = features_dumm; y = target_dumm
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [81]:
GBR = GradientBoostingRegressor()

In [82]:

    parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }

In [83]:
    grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
    grid_GBR.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04],
                         'max_depth': [4, 6, 8, 10],
                         'n_estimators': [100, 500, 1000, 1500],
                         'subsample': [0.9, 0.5, 0.2, 0.1]})

In [96]:
    # print(" Results from Grid Search " )
    # print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
    # print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
    # print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

In [86]:
model = grid_GBR.best_estimator_.fit(features_dumm, target_dumm)
predicted_prices = model.predict(np.array([my_features_dumm]))
print("Предсказанная цена автомобиля =", np.round(predicted_prices[0], 0))

result = sMAPE(target_dumm, pd.DataFrame(model.predict(features_dumm)))
print("Точность предсказания модели:", 100 - round(result, 2), "%")

Предсказанная цена автомобиля = 2079650.0
Точность предсказания модели: 95.59 %


In [88]:
X = features_Ord; y = target_Ord
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [89]:
GBR_Ord = GradientBoostingRegressor()

In [90]:

    parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }

In [91]:
    grid_GBR = GridSearchCV(estimator=GBR_Ord, param_grid = parameters, cv = 2, n_jobs=-1)
    grid_GBR.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04],
                         'max_depth': [4, 6, 8, 10],
                         'n_estimators': [100, 500, 1000, 1500],
                         'subsample': [0.9, 0.5, 0.2, 0.1]})

In [94]:
model = grid_GBR.best_estimator_.fit(features_Ord, target_Ord)
predicted_prices = model.predict(np.array([my_features_Ord]))
print("Предсказанная цена автомобиля =", np.round(predicted_prices[0], 0))

result = sMAPE(target_Ord, pd.DataFrame(model.predict(features_Ord)))
print("Точность предсказания модели:", 100 - round(result, 2), "%")

Предсказанная цена автомобиля = 2024300.0
Точность предсказания модели: 91.0 %


In [99]:
df = pd.DataFrame([{"model": DummyRegressor(strategy='median'), "Точность предсказания, %": 98.26, "Цена авто": 2025000, "Метка кодирования": "any"},
                   {"model": LinearRegression(), "Точность предсказания, %": 89.54, "Цена авто": 2049448, "Метка кодирования": "Ord"},
                   {"model": GradientBoostingRegressor(), "Точность предсказания, %": 95.59, "Цена авто": 2079650, "Метка кодирования": "dumm"}])
df

,model,"Точность предсказания, %",Цена авто,Метка кодирования
0,DummyRegressor(strategy='median'),98.26,2025000,any
1,LinearRegression(),89.54,2049448,Ord
2,GradientBoostingRegressor(),95.59,2079650,dumm


# **Вывод: наиболее ликвидная стоимость продажи автомобиля Tiguan, при точности расчета 98.26% моделью DummyRegressor(strategy='median'), составляет 2025000 рублей.**